In [ ]:
import cvxpy as cp
import numpy as np
import osqp
from scipy import sparse
import time

In [ ]:
def osqp_interface(P,q,A,l,u):
    prob = osqp.OSQP()
    prob.setup(P, q, A, l, u,verbose = False)
    t0 = time.time()
    res = prob.solve()
    return res.x,res.y,time.time() - t0

In [ ]:
ndim = 200
neq = 5
nineq = 5

In [ ]:
P = np.random.random((ndim,ndim))
P = P.T@P+(0.001*np.ones((ndim,ndim)))
Pm = P.copy()
P = sparse.csc_matrix(P)
q = np.random.random(ndim)
A = np.random.random((neq,ndim))
G = np.random.random((nineq,ndim))
b = np.random.random(neq)
h = np.random.random(nineq)
osA = np.vstack([G,A])
osA = sparse.csc_matrix(osA)
l = np.hstack([-np.inf*h,b])
u = np.hstack([h,b])

# 1.OSQP Forward

In [ ]:
x_value, y_value, time_spent = osqp_interface(P,q,osA,l,u)
print('OSQP Forward Time spent:',time_spent)

# 2.OSQP Backward

In [ ]:
lambs = y_value[:nineq] # active set
active_set = np.argwhere(lambs>1e-8)
bG = G[active_set,:].squeeze()
bb = np.zeros(neq)
bh = np.zeros(len(active_set))
bq = np.ones(ndim)
osnewA = np.vstack([bG,A])
osnewA = sparse.csc_matrix(osnewA)
l_new = np.hstack([bh,bb])
u_new = np.hstack([bh,bb])

x_grad, y_grad, time_spent_backward = osqp_interface(P,bq,osnewA,l_new,u_new)
print('OSQP Backward Time spent:',time_spent_backward)

# 3.CVXPY Backward

In [ ]:
qq = cp.Parameter(ndim)
qq.value = q
x1 = cp.Variable(ndim)
prob = cp.Problem(cp.Minimize((1 / 2) * cp.quad_form(x1, P) + qq.T @ x1),
                              [G @ x1 <= h,
                               A @ x1 == b])
t3 = time.time()
prob.solve(requires_grad=True, solver='SCS')
print('CVXPY Forward Time Spent:',time.time() - t3)
t4 = time.time()
prob.backward()
print('CVXPY Backward Time Spent:',time.time() - t4)

# 4. Exact backward (Matrices inverse)

In [ ]:
KKT_L1 = np.hstack([Pm,G.T,A.T])
KKT_L2 = np.hstack([np.diag(lambs)@G, np.diag(G@x_value-h),np.zeros((nineq,neq))])
KKT_L3 = np.hstack([A, np.zeros((neq,neq)),np.zeros((neq,nineq))])
KKT = np.vstack([KKT_L1,KKT_L2,KKT_L3])
t5 = time.time()
exact_bb =-(np.linalg.inv(KKT)@np.hstack([np.ones(ndim),np.zeros(nineq),np.zeros(neq)]))[:ndim]
print("Exact Backward Time Spent", time.time()-t5)

# 6. Accuracy Analysis

In [ ]:
def cal_sse_accuracy(x_exact,x_approx):
    return np.sqrt(np.sum((x_exact - x_approx)**2))

In [ ]:
print('OSQP forward dif',np.sqrt(np.sum((x_value - x1.value)**2)))
print('OSQP backward dif',np.sqrt(np.sum((exact_bb - x_grad)**2)))
print('CVXPY backward dif',np.sqrt(np.sum((exact_bb - qq.gradient)**2)))

# 7. Large scale QPs

In [ ]:
n_list = [10,50,100,500]
nconstraints_list = [5,10,20,50]

In [ ]:
def QP_instances(ndim,neq,nineq):

    P = np.random.random((ndim,ndim))
    P = P.T@P+(0.001*np.ones((ndim,ndim)))
    Pm = P.copy()
    P = sparse.csc_matrix(P)
    q = np.random.random(ndim)
    A = np.random.random((neq,ndim))
    G = np.random.random((nineq,ndim))
    b = np.random.random(neq)
    h = np.random.random(nineq)
    osA = np.vstack([G,A])
    osA = sparse.csc_matrix(osA)
    l = np.hstack([-np.inf*h,b])
    u = np.hstack([h,b])
    return P,q,osA,l,u,Pm

def OSQP_backward(y_value,G):
    nineq,ndim = G.shape
    neq = nineq
    lambs = y_value[:nineq] # active set
    active_set = np.argwhere(lambs>1e-8)
    bG = G[active_set,:].squeeze()
    bb = np.zeros(neq)
    bh = np.zeros(len(active_set))
    bq = np.ones(ndim)
    osnewA = np.vstack([bG,A])
    osnewA = sparse.csc_matrix(osnewA)
    l_new = np.hstack([bh,bb])
    u_new = np.hstack([bh,bb])
    x_grad, y_grad, time_spent_backward = osqp_interface(P,bq,osnewA,l_new,u_new)
    return x_grad, y_grad, time_spent_backward

def cvxpy_backward(P,G,A,h,b,q):
    ndim,nineq = G.shape
    neq = nineq
    qq = cp.Parameter(ndim)
    qq.value = q
    x1 = cp.Variable(ndim)
    prob = cp.Problem(cp.Minimize((1 / 2) * cp.quad_form(x1, P) + qq.T @ x1),
                                  [G @ x1 <= h,
                                   A @ x1 == b])
    t3 = time.time()
    prob.solve(requires_grad=True, solver='SCS')
    time_spent_forward = time.time() - t3
    t4 = time.time()
    prob.backward()
    time_spent_backward = time.time() - t4
    return x1.value,prob.value,time_spent_forward,time_spent_backward

def cal_exact_backward(Pm,G,A,lambs,x_value):
    lambs = y_value[:nineq] # active set
    KKT_L1 = np.hstack([Pm,G.T,A.T])
    KKT_L2 = np.hstack([np.diag(lambs)@G, np.diag(G@x_value-h),np.zeros((nineq,neq))])
    KKT_L3 = np.hstack([A, np.zeros((neq,neq)),np.zeros((neq,nineq))])
    KKT = np.vstack([KKT_L1,KKT_L2,KKT_L3])
    t5 = time.time()
    exact_bb =-(np.linalg.inv(KKT)@np.hstack([np.ones(ndim),np.zeros(nineq),np.zeros(neq)]))[:ndim]
    return exact_bb,time.time()-t5

for ndim,neq in zip(n_list,nconstraints_list):
        P,q,osA,l,u,Pm = QP_instances(ndim,neq,neq) # neq = nineq
        x_value, y_value, time_spent_forward_osqp = osqp_interface(P,q,osA,l,u) # OSQP Forward
        x_grad, y_grad, time_spent_backward_osqp =


